# Assignment

In [1]:
# Import 

import numpy as np
import matplotlib.pyplot as plt
from degree_freedom_queen import *
from degree_freedom_king1 import *
from degree_freedom_king2 import *
from generate_game import *
from Chess_env_new import *


size_board = 4

## The Environment

You can find the environment in the file Chess_env, which contains the class Chess_env. To define an object, you need to provide the board size considered as input. In our example, size_board=4. 
Chess_env is composed by the following methods:

1. Initialise_game. The method initialises an episode by placing the three pieces considered (Agent's king and queen, enemy's king) in the chess board. The outputs of the method are described below in order.

     S $\;$ A matrix representing the board locations filled with 4 numbers: 0, no piece in that position; 1, location of the 
     agent's king; 2 location of the queen; 3 location of the enemy king.
     
     X $\;$ The features, that is the input to the neural network. See the assignment for more information regarding the            definition of the features adopted. To personalise this, go into the Features method of the class Chess_env() and change        accordingly.
     
     allowed_a $\;$ The allowed actions that the agent can make. The agent is moving a king, with a total number of 8                possible actions, and a queen, with a total number of $(board_{size}-1)\times 8$ actions. The total number of possible actions correspond      to the sum of the two, but not all actions are allowed in a given position (movements to locations outside the borders or      against chess rules). Thus, the variable allowed_a is a vector that is one (zero) for an action that the agent can (can't)      make. Be careful, apply the policy considered on the actions that are allowed only.
     

2. OneStep. The method performs a one step update of the system. Given as input the action selected by the agent, it updates the chess board by performing that action and the response of the enemy king (which is a random allowed action in the settings considered). The first three outputs are the same as for the Initialise_game method, but the variables are computed for the position reached after the update of the system. The fourth and fifth outputs are:

     R $\;$ The reward. To change this, look at the OneStep method of the class where the rewards are set.
     
     Done $\;$ A variable that is 1 if the episode has ended (checkmate or draw).
     
     
3. Features. Given the chessboard position, the method computes the features.

This information and a quick analysis of the class should be all you need to get going. The other functions that the class exploits are uncommented and constitute an example on how not to write a python code. You can take a look at them if you want, but it is not necessary.






In [2]:
## INITIALISE THE ENVIRONMENT

env=Chess_Env_new(size_board)

### Random Agent

In [3]:
## PRINT 5 STEPS OF AN EPISODE CONSIDERING A RANDOM AGENT

S,X,allowed_a=env.Initialise_game(new_feature_k1=False,new_feature_q1=False)                       # INTIALISE GAME

print(S)                                                  # PRINT CHESS BOARD (SEE THE DESCRIPTION ABOVE)

print('check? ',env.check)                                # PRINT VARIABLE THAT TELLS IF ENEMY KING IS IN CHECK (1) OR NOT (0)
print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))    # PRINT THE NUMBER OF LOCATIONS THAT THE ENEMY KING CAN MOVE TO


for i in range(5):
    
    a,_=np.where(allowed_a==1)                  # FIND WHAT THE ALLOWED ACTIONS ARE
    a_agent=np.random.permutation(a)[0]         # MAKE A RANDOM ACTION
    print("action",a_agent)
    S,X,allowed_a,R,Done=env.OneStep(a_agent,new_reward=False,new_feature_k1=False,new_feature_q1=False)   # UPDATE THE ENVIRONMENT
    
    ## PRINT CHESS BOARD AND VARIABLES
    print('')
    print("S",S)
    print(R,'', Done)
    print('check? ',env.check)
    print('dofk2 ',np.sum(env.dfk2_constrain).astype(int))
    
    
    # TERMINATE THE EPISODE IF Done=True (DRAW OR CHECKMATE)
    if Done:
        break



[[0 0 0 0]
 [0 0 0 2]
 [3 0 0 1]
 [0 0 0 0]]
check?  0
dofk2  2
action 31

S [[0 0 0 0]
 [0 0 1 2]
 [0 0 0 0]
 [3 0 0 0]]
0  0
check?  0
dofk2  1
action 0

S [[0 0 0 0]
 [0 0 1 0]
 [0 0 0 2]
 [0 3 0 0]]
0  0
check?  0
dofk2  1
action 3

S [[0 0 0 0]
 [0 0 1 2]
 [0 0 0 0]
 [0 0 3 0]]
0  0
check?  0
dofk2  0
action 3

S [[0 0 0 2]
 [0 0 1 0]
 [0 0 0 0]
 [0 3 0 0]]
0  0
check?  0
dofk2  3
action 1

S [[0 0 0 0]
 [0 0 1 0]
 [0 0 0 2]
 [3 0 0 0]]
0  0
check?  0
dofk2  1


In [4]:
# PERFORM N_episodes=1000 EPISODES MAKING RANDOM ACTIONS AND COMPUTE THE AVERAGE REWARD AND NUMBER OF MOVES 

S,X,allowed_a=env.Initialise_game(new_feature_k1=False,new_feature_q1=False)
N_episodes=1000

# VARIABLES WHERE TO SAVE THE FINAL REWARD IN AN EPISODE AND THE NUMBER OF MOVES 
R_save_random = np.zeros([N_episodes, 1])
N_moves_save_random = np.zeros([N_episodes, 1])

for n in range(N_episodes):
    
    S,X,allowed_a=env.Initialise_game(new_feature_k1=False,new_feature_q1=False)     # INITIALISE GAME
    Done=0                                  # SET Done=0 AT THE BEGINNING
    i=1                                     # COUNTER FOR THE NUMBER OF ACTIONS (MOVES) IN AN EPISODE
    
    # UNTIL THE EPISODE IS NOT OVER...(Done=0)
    while Done==0:
        
        # SAME AS THE CELL BEFORE, BUT SAVING THE RESULTS WHEN THE EPISODE TERMINATES 
        
        a,_=np.where(allowed_a==1)
        a_agent=np.random.permutation(a)[0]
        S,X,allowed_a,R,Done=env.OneStep(a_agent,new_reward=False,new_feature_k1=False,new_feature_q1=False)        
        
        if Done:
            
            R_save_random[n]=np.copy(R)
            N_moves_save_random[n]=np.copy(i)

            break

        i=i+1                               # UPDATE THE COUNTER



# AS YOU SEE, THE PERFORMANCE OF A RANDOM AGENT ARE NOT GREAT, SINCE THE MAJORITY OF THE POSITIONS END WITH A DRAW 
# (THE ENEMY KING IS NOT IN CHECK AND CAN'T MOVE)

print('Random_Agent, Average reward:',np.mean(R_save_random),'Number of steps: ',np.mean(N_moves_save_random))



Random_Agent, Average reward: 0.18 Number of steps:  7.044


## Deep RL Agent

### Initialize NN Parameters

In [5]:
# INITIALISE THE PARAMETERS OF YOUR NEURAL NETWORK AND...
# PLEASE CONSIDER TO USE A MASK OF ONE FOR THE ACTION MADE AND ZERO OTHERWISE IF YOU ARE NOT USING VANILLA GRADIENT DESCENT...
# WE SUGGEST A NETWORK WITH ONE HIDDEN LAYER WITH SIZE 200. 

import numpy.matlib 
S,X,allowed_a=env.Initialise_game(new_feature_k1=False,new_feature_q1=False)
N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
N_in=np.shape(X)[0]    ## INPUT SIZE
N_h=200                ## NUMBER OF HIDDEN NODES

#set random seed
np.random.seed(10)
## INITALISE YOUR NEURAL NETWORK...


# HYPERPARAMETERS SUGGESTED (FOR A GRID SIZE OF 4)

epsilon_0 = 0.2     # STARTING VALUE OF EPSILON FOR THE EPSILON-GREEDY POLICY
beta = 0.0005      # THE PARAMETER SETS HOW QUICKLY THE VALUE OF EPSILON IS DECAYING (SEE epsilon_f BELOW)
gamma = 0.85        # THE DISCOUNT FACTOR
eta = 0.0035        # THE LEARNING RATE

N_episodes = 100000 # THE NUMBER OF GAMES TO BE PLAYED 

# SAVING VARIABLES
R_save = np.zeros([N_episodes, 1])
N_moves_save = np.zeros([N_episodes, 1])

#weight initialise
#W1 = np.random.randn(N_in, N_h)/1000
#W2 = np.random.randn(N_h,N_a)/1000

W1 = np.random.uniform(0,1,(N_h, N_in))
W2 = np.random.uniform(0,1,(N_a, N_h))

# The following normalises the random weights so that the sum of each row =1
W1 = np.divide(W1,np.matlib.repmat(np.sum(W1,1)[:,None],1,N_in))
W2 = np.divide(W2,np.matlib.repmat(np.sum(W2,1)[:,None],1,N_h))

bias_W1 = np.zeros((N_h,))
bias_W2 = np.zeros((N_a,))


print("W",W1,W2)
print("bias",np.shape(bias_W1),bias_W1,bias_W2)
x1=np.dot(W1,X)+bias_W1
print("X",np.shape(X),np.shape(x1))
print(np.shape(np.dot(W2,x1)+bias_W2))

W [[0.0263052  0.00070773 0.02161    ... 0.03220316 0.03368208 0.01556186]
 [0.03147359 0.00957684 0.02275864 ... 0.00606496 0.0019231  0.0128701 ]
 [0.0038591  0.00638885 0.03163403 ... 0.03030136 0.02153973 0.02789269]
 ...
 [0.00745545 0.03119801 0.01999057 ... 0.02399051 0.02606457 0.00365012]
 [0.00074568 0.02364644 0.01050774 ... 0.01110022 0.01216128 0.02496483]
 [0.01577329 0.01397078 0.02888768 ... 0.02235599 0.00235686 0.02845395]] [[0.00435651 0.00924724 0.00462405 ... 0.00140784 0.00832301 0.00720998]
 [0.0098536  0.00586077 0.00616222 ... 0.0010806  0.00319171 0.0024448 ]
 [0.00129827 0.00909055 0.00610403 ... 0.00563997 0.00870195 0.00406107]
 ...
 [0.00015967 0.00999858 0.00533841 ... 0.0044824  0.00196832 0.00286865]
 [0.00964089 0.00504781 0.00054623 ... 0.00339875 0.00193406 0.00770252]
 [0.00671708 0.00426894 0.00408619 ... 0.00791993 0.00779112 0.00796532]]
bias (200,) [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 

### EpsilonGreedy_Policy

In [6]:
def EpsilonGreedy_Policy(Qvalues, epsilon, allowed_a):
    
    N_a=np.shape(Qvalues)[0]

    rand_value=np.random.uniform(0,1)

    rand_a=rand_value<epsilon
    a=np.where(allowed_a==1)[0]
    Qvalues_allowed=np.copy(Qvalues[a])
    if rand_a==True:
        
        a_agent=np.random.permutation(a)[0]

    else:
        '''
        sort_a=np.argsort(Qvalues)[::-1]
        while a_item in sort_a:
            if a_item in a:
                return a_item
        '''
        a1=np.argmax(Qvalues_allowed)
        a_agent=np.copy(a[a1])
            
    return a_agent

### Exponential moving average

In [7]:
def ema(lists, window):
    alpha = 2/(window+1.0)
    new_list = np.zeros(len(lists))
    new_list[0] = (lists[0]+lists[1])/2
    for i in range(1, len(lists)):
        new_list[i] = alpha*lists[i] + (1-alpha)*new_list[i-1]
    return new_list

# SARSA

In [8]:

# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...
def SARSA(epsilon_0,beta,gamma,new_reward,new_feature_k1,new_feature_q1):
    #env=Chess_Env(size_board)
    S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)
    N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
    N_in=np.shape(X)[0]    ## INPUT SIZEs
    N_h=200                ## NUMBER OF HIDDEN NODES
    np.random.seed(5001)
    
    R_save = []
    N_moves_save=[]
  
    W1 = np.random.randn(N_h, N_in) * np.sqrt(1 / (N_in))
    W2 = np.random.randn(N_a, N_h) * np.sqrt(1 / (N_h))
    bias_W1 = np.zeros((N_h,))
    bias_W2 = np.zeros((N_a,))

    best_R=0
    best_episode=0
    
    R_mean_list=[]
    N_moves_save_mean=[]
    for n in range(N_episodes):

        epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
        Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS

        S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)      ## INITIALISE GAME
        print("N_episodes",n)                    ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING

        h1 = np.dot(W1,X)+bias_W1

        # Apply the sigmoid function
        x1 = 1/(1+np.exp(-h1))

        Qvalues=np.dot(W2,x1)+bias_W2
        Qvalues = 1/(1+np.exp(-Qvalues))
        
        print(Qvalues)    
        a_agent = EpsilonGreedy_Policy(Qvalues, epsilon_f, allowed_a)
        while Done==0:                           ## START THE EPISODE


           
            S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent,new_reward,new_feature_k1,new_feature_q1)
            #print("S",S,"action",a_agent,R,Done)

            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done==1:
                

                # Update the Qvalues
                delta=(R-Qvalues[a_agent])*Qvalues[a_agent]*(1-Qvalues[a_agent])
                
                W2[a_agent,:]=W2[a_agent,:]+eta*np.outer(delta,x1)
                bias_W2[a_agent] = bias_W2[a_agent]+eta*delta

                delta1 =  np.dot(W2[a_agent,:], delta)*x1*(1-x1)
            
                W1 += eta* np.outer(delta1,X.T)

                bias_W1 += eta*delta1
                
                #save the R_save,N_moves_save
                R_save.append(R)
                print(R,np.mean(R_save),best_R)
                R_mean_list.append(np.mean(R_save))
                if np.mean(R_save)>best_R:
                    best_R=np.mean(R_save)
                    best_episode=n
                
                N_moves_save.append(i)
                N_moves_save_mean.append(np.mean(N_moves_save))
                break

            # IF THE EPISODE IS NOT OVER...
            h1_next = np.dot(W1,X_next)+bias_W1

            # Apply the sigmoid function
            x1_next = 1/(1+np.exp(-h1_next))           
            
            Qvalues2=np.dot(W2,x1_next)+bias_W2
            Qvalues2 = 1/(1+np.exp(-Qvalues2))
            
            a_pri = EpsilonGreedy_Policy(Qvalues2, epsilon_f, allowed_a_next)
            
            # Compute the delta
            delta=(R+gamma*Qvalues2[a_pri]-Qvalues[a_agent])*Qvalues[a_agent]*(1-Qvalues[a_agent])
            
            # Update the weights
            W2[a_agent,:]=W2[a_agent,:]+eta*np.outer(delta,x1)
            #print("W2,delta,x",np.shape(W2),np.shape(delta),np.shape(Qvalues2))
            bias_W2[a_agent] = bias_W2[a_agent]+eta*delta

            delta1 =  np.dot(W2[a_agent,:], delta)*x1*(1-x1)
            
            W1 += eta* np.outer(delta1,X)

            bias_W1 += eta*delta1
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE...     
            S=np.copy(S_next)
            X=np.copy(X_next)
            a_agent=a_pri
            allowed_a=np.copy(allowed_a_next)
            Qvalues=np.copy(Qvalues2)
            x1=x1_next
            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
            #print("action i",i)
        
    return R_save,N_moves_save,(best_R,best_episode),R_mean_list,N_moves_save_mean

In [ ]:
N_episodes = 100000
R_save,N_moves_save,(best_R,best_episode),R_mean_list,N_moves_save_mean = SARSA(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
print('SARSA_Agent, Average reward:',np.mean(R_save),'Number of steps: ',np.mean(N_moves_save))

N_episodes 0
[0.55920816 0.49827056 0.53642625 0.61660214 0.66958994 0.48295781
 0.49655108 0.61387965 0.5808479  0.65167749 0.42441025 0.49593275
 0.63809396 0.72860296 0.53270269 0.49152841 0.39753577 0.7512336
 0.64600537 0.6027111  0.53024397 0.65460882 0.40737854 0.48964167
 0.57989335 0.40226498 0.36415316 0.56068206 0.71529414 0.6377625
 0.56775309 0.47102008]
1 1.0 0
N_episodes 1
[0.53837112 0.53719111 0.51730225 0.65109431 0.64575376 0.52424803
 0.48442178 0.57364257 0.55645922 0.66829641 0.4199311  0.49635811
 0.65439542 0.74045385 0.52623082 0.52238135 0.41840646 0.749343
 0.65495426 0.60265345 0.49032322 0.61296974 0.37171886 0.47456443
 0.57442987 0.41095179 0.37737292 0.57231383 0.70843572 0.64000245
 0.5710684  0.46967731]
0.0 0.5 1.0
N_episodes 2
[0.53459269 0.54273389 0.53669078 0.65641464 0.64917126 0.5093881
 0.50763164 0.59532589 0.58720035 0.64426482 0.44220075 0.50493042
 0.63370273 0.74932737 0.53665779 0.48731981 0.4065326  0.73946358
 0.60730062 0.61508868 0.48

1 0.1 1.0
N_episodes 30
[0.55338414 0.50225414 0.53757219 0.61000322 0.63568324 0.49637491
 0.50986758 0.59667419 0.58552664 0.60650257 0.44759073 0.50137124
 0.62916311 0.72200505 0.53888955 0.5089222  0.37489858 0.7398883
 0.59441019 0.58219797 0.48635464 0.60147243 0.41990156 0.45869307
 0.58822693 0.40159991 0.38082628 0.55700241 0.70617764 0.63640429
 0.54780033 0.48405626]
0.0 0.0967741935483871 1.0
N_episodes 31
[0.52527014 0.53346843 0.51923117 0.62063832 0.6246313  0.51545458
 0.50648354 0.56969784 0.56636072 0.58250517 0.44066993 0.49444841
 0.64227741 0.73629514 0.5304935  0.52793036 0.3856935  0.72687099
 0.60018641 0.5657117  0.54784218 0.60128543 0.40043912 0.49138529
 0.57811286 0.42375528 0.37619159 0.5745528  0.71160344 0.63273341
 0.55806732 0.44138511]
0.0 0.09375 1.0
N_episodes 32
[0.5576337  0.54433701 0.55525707 0.65164298 0.61427873 0.51602136
 0.49994419 0.57696164 0.58030485 0.59310571 0.3962158  0.48842274
 0.60734607 0.71994794 0.53395587 0.5103753  0.4006987

0.0 0.05454545454545454 1.0
N_episodes 55
[0.52641384 0.51330419 0.57009999 0.56522851 0.57131875 0.52080466
 0.49058962 0.54920736 0.56426196 0.54673446 0.41209794 0.49867489
 0.57604114 0.68311822 0.53668118 0.49481111 0.41165263 0.6780175
 0.57950427 0.60402721 0.54408435 0.58514276 0.38869246 0.43696276
 0.58050269 0.40201945 0.36915709 0.57737489 0.67160146 0.58329147
 0.55254565 0.48986916]
0.0 0.05357142857142857 1.0
N_episodes 56
[0.55376926 0.51082301 0.53820702 0.58708635 0.56966893 0.50979498
 0.49115429 0.58093457 0.55072304 0.57192921 0.45949914 0.51651901
 0.62311015 0.67497695 0.51652366 0.49587189 0.39456885 0.70554014
 0.57046162 0.61797363 0.52240626 0.55671213 0.39186192 0.47596145
 0.56613204 0.36955745 0.37413372 0.56155645 0.65218462 0.58993195
 0.54173282 0.48748471]
0.0 0.05263157894736842 1.0
N_episodes 57
[0.51676969 0.49118462 0.54834549 0.57834467 0.57638828 0.49802484
 0.49872012 0.55609614 0.55398059 0.53596233 0.44337744 0.51942005
 0.59161178 0.67191277 

0.0 0.06172839506172839 1.0
N_episodes 81
[0.49254643 0.5185031  0.50579283 0.55155408 0.54911658 0.52013706
 0.50267927 0.54831886 0.56519441 0.52817248 0.43825367 0.51050873
 0.58456475 0.70014496 0.51188619 0.51653859 0.39365086 0.66573891
 0.51813036 0.56444078 0.53924454 0.51357907 0.42197632 0.48138873
 0.56748118 0.42873215 0.37536807 0.5461337  0.65286961 0.58356119
 0.53319335 0.46097919]
0.0 0.06097560975609756 1.0
N_episodes 82
[0.57099003 0.52549578 0.51478307 0.55758033 0.53628704 0.51038111
 0.46947866 0.55420613 0.56692042 0.54019701 0.43264151 0.49602237
 0.60151412 0.66851955 0.5265763  0.52398637 0.36517052 0.6902259
 0.5408292  0.59745964 0.53414434 0.51207956 0.39546789 0.50141614
 0.55904514 0.40915166 0.37210143 0.52680154 0.6327818  0.57878673
 0.5407104  0.47589599]
0.0 0.060240963855421686 1.0
N_episodes 83
[0.56212778 0.51161219 0.55496919 0.56695331 0.52981155 0.50772559
 0.49743906 0.58566866 0.58063159 0.51855291 0.39425413 0.49399654
 0.55605287 0.65675232

0.0 0.06666666666666667 1.0
N_episodes 105
[0.49480928 0.51848042 0.50640518 0.50768987 0.48449379 0.50480045
 0.49389081 0.51542704 0.56205309 0.52045851 0.41970773 0.49216932
 0.53521393 0.66032921 0.49989071 0.50111056 0.39472445 0.63899725
 0.53001697 0.56122564 0.49792222 0.48124233 0.4050346  0.46698678
 0.54197463 0.39910128 0.38009429 0.49736048 0.58846907 0.54825415
 0.52269916 0.46234737]
0.0 0.0660377358490566 1.0
N_episodes 106
[0.49587431 0.51984894 0.50339723 0.50989958 0.53782673 0.52115343
 0.48995402 0.53121417 0.57138371 0.50231432 0.41457937 0.51030002
 0.52080695 0.66874435 0.53168437 0.51165878 0.39903041 0.63444315
 0.53917417 0.55809294 0.4920489  0.49229262 0.39942103 0.48223827
 0.56198604 0.4196707  0.38125677 0.49221902 0.6042336  0.53535406
 0.52315852 0.46471257]
0.0 0.06542056074766354 1.0
N_episodes 107
[0.50336522 0.51256449 0.54836125 0.49768484 0.51307955 0.49980251
 0.50932426 0.54604516 0.56511446 0.53222661 0.4378364  0.5039176
 0.49508249 0.6723978

...

1 0.5982380319148937 1.0
N_episodes 6016
[0.25967301 0.19620554 0.350296   0.20575861 0.32879725 0.20561016
 0.29733051 0.33277888 0.20966274 0.1712918  0.21220855 0.43404558
 0.18180618 0.1845713  0.21496066 0.18008489 0.25159442 0.24389415
 0.1855267  0.37014595 0.38396884 0.18624065 0.19243269 0.32986787
 0.17002179 0.16116956 0.18530059 0.17768402 0.19011369 0.16397163
 0.1910402  0.238933  ]
1 0.5983048030580024 1.0
N_episodes 6017
[0.26335201 0.17456859 0.35984865 0.22744442 0.32951069 0.18710208
 0.27549335 0.29866377 0.21138921 0.17535059 0.20433916 0.4279776
 0.18125859 0.1944788  0.22512274 0.18620974 0.25096549 0.22657033
 0.17687233 0.34108764 0.39345496 0.18370785 0.20735428 0.31956351
 0.17417841 0.19347614 0.17268089 0.1766225  0.17061971 0.15255413
 0.19052694 0.25978671]
0.0 0.5982053838484547 1.0
N_episodes 6018
[0.26753278 0.18572627 0.36867929 0.22782809 0.31487337 0.20236227
 0.29375687 0.31139536 0.22856699 0.18780837 0.1746915  0.39777621
 0.17141676 0.19647309 0

1 0.5990402118153235 1.0
N_episodes 6043
[0.27195442 0.19029908 0.35187619 0.20870089 0.30053556 0.20182559
 0.30845379 0.35247078 0.22793796 0.18238853 0.21273563 0.42944969
 0.17980909 0.19307516 0.21674381 0.18603498 0.24390173 0.24594251
 0.17731516 0.3706297  0.3792424  0.18566215 0.18346042 0.33485255
 0.17382883 0.16365185 0.18163355 0.17531446 0.19476063 0.16503453
 0.18708854 0.25213671]
1 0.5991065519523494 1.0
N_episodes 6044
[0.24880651 0.18452252 0.36602528 0.2237354  0.30995188 0.21207389
 0.3212206  0.33227574 0.22694472 0.18635958 0.19881101 0.3836637
 0.17336799 0.19385265 0.22036156 0.19251456 0.25646613 0.24864805
 0.18363527 0.35092885 0.34415421 0.16588009 0.18247479 0.30754465
 0.17463928 0.18113083 0.19115669 0.17542765 0.18497569 0.14841185
 0.17279092 0.24245052]
1 0.599172870140612 1.0
N_episodes 6045
[0.25871485 0.19620062 0.35229846 0.20647278 0.31465177 0.20561693
 0.31064811 0.35288217 0.20966966 0.17129428 0.21220761 0.45027065
 0.1817947  0.18457425 0.21

1 0.6000658870037885 1.0
N_episodes 6071
[0.24528598 0.18043879 0.37931439 0.22678975 0.29496838 0.18942118
 0.27841807 0.32983503 0.20089951 0.17296518 0.21714985 0.4504123
 0.19751052 0.19757542 0.19585866 0.17534811 0.24636701 0.23386617
 0.18590948 0.37267032 0.38090511 0.17793577 0.19243413 0.32006418
 0.17066636 0.1621815  0.18147102 0.17562946 0.17675338 0.14702751
 0.18759336 0.23508426]
0.0 0.5999670619235836 1.0
N_episodes 6072
[0.26534355 0.18431722 0.38207193 0.2364327  0.30445636 0.190362
 0.30312092 0.36021545 0.22157576 0.17038316 0.19823539 0.41167476
 0.16663233 0.19430228 0.21049455 0.17065272 0.25926794 0.2513168
 0.17301242 0.37569891 0.3381062  0.17285529 0.19969141 0.31702159
 0.16850894 0.16353532 0.17023558 0.15851392 0.17174901 0.1551491
 0.1774597  0.24315824]
1 0.6000329326527252 1.0
N_episodes 6073
[0.27884634 0.18246095 0.38216108 0.21243166 0.31533151 0.20545563
 0.31539156 0.36457791 0.23654688 0.20149941 0.21506164 0.41468468
 0.18844994 0.19403468 0.216

0.0 0.600885100803147 1.0
N_episodes 6101
[0.31399873 0.17936987 0.37583549 0.21333761 0.32574821 0.19868178
 0.3161525  0.37161823 0.25237483 0.19693068 0.21112375 0.43892004
 0.18705564 0.18890487 0.22194673 0.19910459 0.21582503 0.2772638
 0.17918541 0.3581475  0.35341328 0.18005733 0.20958394 0.32734417
 0.18659676 0.17578364 0.18179842 0.1621615  0.19002317 0.17039307
 0.19183483 0.26888745]
1 0.600950508030154 1.0
N_episodes 6102
[0.26985511 0.18345232 0.35464297 0.22329565 0.30833546 0.20633652
 0.31450118 0.33450707 0.23445885 0.18032849 0.21550924 0.4742525
 0.19583985 0.19696139 0.20332476 0.2022152  0.22475237 0.24523537
 0.20701244 0.35485642 0.3356699  0.17109146 0.19380781 0.34093962
 0.17125769 0.17790104 0.18455701 0.16436935 0.18106926 0.15862233
 0.20859789 0.26080457]
1 0.6010158938227101 1.0
N_episodes 6103
[0.29178576 0.18918891 0.36911391 0.20605958 0.30323114 0.19914498
 0.31556729 0.35103621 0.22138681 0.19378985 0.202882   0.46250266
 0.17045903 0.19399759 0.22

[0.28581713 0.19060727 0.38157483 0.20877984 0.30872615 0.20183353
 0.31727536 0.35496493 0.22794948 0.18272604 0.2131401  0.4683505
 0.17980512 0.19307058 0.21675263 0.18584652 0.24333917 0.25035704
 0.17731438 0.38398973 0.38385409 0.18602322 0.18345684 0.36353356
 0.17404947 0.16447067 0.18196042 0.17686615 0.19549448 0.16524295
 0.18709839 0.2586122 ]
1 0.6021540469973891 1.0
N_episodes 6128
[0.3014502  0.17518172 0.3827485  0.20750003 0.32455739 0.21030777
 0.32751232 0.36083121 0.23342819 0.20941176 0.20364209 0.4583401
 0.18664809 0.19269503 0.21391584 0.18419397 0.24124053 0.26478448
 0.17622005 0.3824537  0.36582023 0.19069281 0.19549407 0.36735971
 0.17951927 0.17493968 0.18943415 0.1829183  0.18326408 0.17350014
 0.17861626 0.28188145]
0.0 0.6020558002936858 1.0
N_episodes 6129
[0.30009006 0.19109649 0.39923644 0.22642775 0.2970023  0.19425527
 0.3128649  0.32171361 0.24472535 0.1908443  0.19479375 0.4262952
 0.18227115 0.17604855 0.20188125 0.19786075 0.22606878 0.24152811


In [ ]:
plt.figure()
plt.title('Sarsa Reward Mean')
plt.plot(np.arange(100,N_episodes+1,1),R_mean_list[99:])
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()
#print(R_mean_list)

In [ ]:
plt.figure()
plt.title('Sarsa Steps Mean')
plt.plot(np.arange(100,N_episodes+1,1),N_moves_save_mean[99:])
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

In [ ]:
R_list=ema(R_save,1000)
N_list=ema(N_moves_save,1000)
print(np.shape(R_list))

In [ ]:
plt.figure()
plt.title('Sarsa Reward Ema')
plt.plot(np.arange(0,N_episodes,1),R_list)
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()
#print(R_mean_list)

In [ ]:
plt.figure()
plt.title('Sarsa Steps Ema')
plt.plot(np.arange(0,N_episodes,1),N_list)
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

## Adam

In [ ]:
## Let's define Adam, see notebook Adam's playgroud for more information

class Adam:

    def __init__(self, Params, beta1):
        
        N_dim=np.shape(np.shape(Params))[0] # It finds out if the parameters given are in a vector (N_dim=1) or a matrix (N_dim=2)
        
        # INITIALISATION OF THE MOMENTUMS
        if N_dim==1:
               
            self.N1=np.shape(Params)[0]
            
            self.mt=np.zeros([self.N1])
            self.vt=np.zeros([self.N1])
        
        if N_dim==2:
            
            self.N1=np.shape(Params)[0]
            self.N2=np.shape(Params)[1]
        
            self.mt=np.zeros([self.N1,self.N2])
            self.vt=np.zeros([self.N1,self.N2])
        
        # HYPERPARAMETERS OF ADAM
        self.beta1=beta1
        self.beta2=0.999
        
        self.epsilon=10**(-8)
        
        # COUNTER OF THE TRAINING PROCESS
        self.counter=0
        
        
    def Compute(self,Grads):
                
        self.counter=self.counter+1
        
        self.mt=self.beta1*self.mt+(1-self.beta1)*Grads
        
        self.vt=self.beta2*self.vt+(1-self.beta2)*Grads**2
        
        mt_n=self.mt/(1-self.beta1**self.counter)
        vt_n=self.vt/(1-self.beta2**self.counter)
        
        New_grads=mt_n/(np.sqrt(vt_n)+self.epsilon)
        
        return New_grads

beta1=0.9 # First order momentum for Adam

### SARSA with Adam

In [ ]:
def SARSA_Adam(epsilon_0,beta,gamma,new_reward,new_feature_k1,new_feature_q1):
    #env=Chess_Env(size_board)
    S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)
    N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
    N_in=np.shape(X)[0]    ## INPUT SIZE
    N_h=200                ## NUMBER OF HIDDEN NODES
    np.random.seed(101)
    
    # R_save_A = np.zeros([N_episodes, 1])
    R_save_A = []
    # N_moves_save_A = np.zeros([N_episodes, 1])
    N_moves_save_A = []
    W1_A = np.random.uniform(0,1,(N_h, N_in))
    W2_A = np.random.uniform(0,1,(N_a, N_h))

            # The following normalises the random weights so that the sum of each row =1
    W1_A = np.random.randn(N_h, N_in) * np.sqrt(1 / (N_in))
    W2_A = np.random.randn(N_a, N_h) * np.sqrt(1 / (N_h))
    bias_W1_A = np.zeros((N_h,))
    bias_W2_A = np.zeros((N_a,))
    
    # Intialise Adam for the parameters
    Adam_W1=Adam(W1_A,beta1)
    Adam_W2=Adam(W2_A,beta1)
    Adam_bias_W1=Adam(bias_W1_A,beta1)
    Adam_bias_W2=Adam(bias_W2_A,beta1)
    
    
    R_mean_list_A=[]
    N_moves_save_mean_A=[]
    
    best_R_A = 0
    best_episode_A = 0
    for n in range(N_episodes):

        epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
        Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS

        S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)      ## INITIALISE GAME
        print("N_episodes",n)                    ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING

        h1 = np.dot(W1_A,X)+bias_W1_A

        # Apply the sigmoid function
        x1 = 1/(1+np.exp(-h1))
            
        Qvalues=np.dot(W2_A,x1)+bias_W2_A
        Qvalues = 1/(1+np.exp(-Qvalues))
        
        print(Qvalues)    
        a_agent = EpsilonGreedy_Policy(Qvalues, epsilon_f, allowed_a)
        while Done==0:                           ## START THE EPISODE


            ## THIS IS A RANDOM AGENT, CHANGE IT to SARSA with value function approximation

            S_next,X_next,allowed_a_next,R,Done=env.OneStep(a_agent,new_reward,new_feature_k1,new_feature_q1)


            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done==1:
                # Update the Qvalues
                delta=(R-Qvalues[a_agent])*Qvalues[a_agent]*(1-Qvalues[a_agent])

                W2_A[a_agent,:]=W2_A[a_agent,:]+eta*Adam_W2.Compute(np.outer(delta,x1))[a_agent,:]
                bias_W2_A[a_agent] = bias_W2[a_agent]+eta*Adam_bias_W2.Compute(delta)[a_agent]

                delta1 =  np.dot(W2_A[a_agent,:], delta)*x1*(1-x1)
                
                W1_A += eta* Adam_W1.Compute(np.outer(delta1,X.T))

                bias_W1_A += eta*Adam_bias_W1.Compute(delta1)
                
                R_save_A.append(R)
                print(R,np.mean(R_save_A),best_R_A)
                R_mean_list_A.append(np.mean(R_save_A))
                if np.mean(R_save_A)>best_R_A:
                    best_R_A=np.mean(R_save_A)
                    best_episode_A=n
                    
                N_moves_save_A.append(i)
                N_moves_save_mean_A.append(np.mean(N_moves_save_A))
                break

            # IF THE EPISODE IS NOT OVER...
            h1_next = np.dot(W1_A,X_next)+bias_W1_A

            # Apply the sigmoid function
            x1_next = 1/(1+np.exp(-h1_next))
            
            Qvalues2=np.dot(W2_A,x1_next)+bias_W2_A
            Qvalues2 = 1/(1+np.exp(-Qvalues2))
            
            a_pri = EpsilonGreedy_Policy(Qvalues2, epsilon_f, allowed_a_next)
            
            # Compute the delta
            
            delta=(R+gamma*Qvalues2[a_pri]-Qvalues[a_agent])*Qvalues[a_agent]*(1-Qvalues[a_agent])\
            
            # Update the weights

            W2_A[a_agent,:]=W2_A[a_agent,:]+eta*Adam_W2.Compute(np.outer(delta,x1))[a_agent,:]
            
            bias_W2_A[a_agent] = bias_W2_A[a_agent]+eta*Adam_bias_W2.Compute(delta)[a_agent]

            
            delta1 =  np.dot(W2_A[a_agent,:], delta)*x1*(1-x1)
            
            W1_A += eta* Adam_W1.Compute(np.outer(delta1,X))

            bias_W1_A += eta*Adam_bias_W1.Compute(delta1)
            
           # print("Qvalue",Qvalues2)
            # NEXT STATE AND CO. BECOME ACTUAL STATE...     
            S=np.copy(S_next)
            X=np.copy(X_next)
            a_agent=a_pri
            allowed_a=np.copy(allowed_a_next)
            Qvalues=np.copy(Qvalues2)
            x1=x1_next
            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS
            #print("action i",i)
    return R_save,N_moves_save,(best_R,best_episode),R_mean_list,N_moves_save_mean

In [ ]:
N_episodes = 100000
R_save_A,N_moves_save_A,(best_R_A,best_episode_A),R_mean_list_A,N_moves_save_mean_A = SARSA_Adam(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
print(np.mean(R_save_A),np.mean(N_moves_save_A))

In [ ]:
plt.figure()
plt.title('SARSA Adam Reward Mean')
plt.plot(np.arange(100,N_episodes+1,1),R_mean_list_A[99:])
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()
#print(R_mean_list)

In [ ]:
plt.figure()
plt.title('SARSA Adam Steps Mean')
plt.plot(np.arange(100,N_episodes+1,1),N_moves_save_mean_A[99:])
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

In [ ]:
R_list_A=ema(R_save_A,1000)
N_list_A=ema(N_moves_save_A,1000)
print(np.shape(R_list_A))

In [ ]:
plt.figure()
plt.title('SARSA Adam Reward Ema')
plt.plot(np.arange(0,N_episodes,1),R_list_A)
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()
#print(R_mean_list)

In [ ]:
plt.figure()
plt.title('SARSA Adam Steps Ema')
plt.plot(np.arange(0,N_episodes,1),N_list_A)
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

In [ ]:
N_episodes = 30000
beta_list = [0.00001,0.00005,0.0001]     
gamma_list = [0.8,0.85,0.9]
result_list = []
for beta in beta_list:
    print("BETA",beta)
    for gamma in gamma_list:
        R_save,N_moves_save,(best_R,best_episode),R_mean_list,N_moves_save_mean = SARSA_Adam(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
        result_list.append(((epsilon_0,beta,gamma),(np.mean(R_save),np.mean(N_moves_save), R_save,N_moves_save,R_mean_list,N_moves_save_mean)))
        print('SARSA_Agent, Average reward:',np.mean(R_save),'\nNumber of steps: ',np.mean(N_moves_save),
              '\nparameters: epsilon:',epsilon_0,'beta:',beta,'gamma:',gamma)

In [ ]:
print(np.mean(N_moves_save),np.mean(R_save))

## Q-Learning

In [ ]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...
def Q_Learning(epsilon_0,beta,gamma,new_reward,new_feature_k1,new_feature_q1):
    S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)
    
    N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
    N_in=np.shape(X)[0]    ## INPUT SIZE
    N_h=200                ## NUMBER OF HIDDEN NODES
    np.random.seed(101)
    
    R_save = np.zeros([N_episodes, 1])
    N_moves_save = np.zeros([N_episodes, 1])

    W1 = np.random.randn(N_h, N_in) * np.sqrt(1 / (N_in))
    W2 = np.random.randn(N_a, N_h) * np.sqrt(1 / (N_h))
    
    bias_W1 = np.zeros((N_h,))
    bias_W2 = np.zeros((N_a,))

    for n in range(N_episodes):

        epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
        Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS

        S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)      ## INITIALISE GAME
        print("N_episodes",n)                    ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING

        
        
        while Done==0:                           ## START THE EPISODE

            h1 = np.dot(W1, X) + bias_W1
            x1 = 1/(1+np.exp(-h1))
            h2 = np.dot(W2, x1)+bias_W2
            Qvalues = 1/(1+np.exp(-h2))

            a_agent = EpsilonGreedy_Policy(Qvalues, epsilon_f, allowed_a)

            S_next, X_next, allowed_a_next, R, Done = env.OneStep(a_agent,new_reward,new_feature_k1,new_feature_q1)
#             print("S",S,"action",a_agent, "R", R)

            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done==1:
                delta = R-Qvalues[a_agent]
                                
                # Update the weights
                gd2 = delta*Qvalues*(1-Qvalues)
                W2[a_agent,:] = W2[a_agent,:] + eta*np.outer(gd2, x1)[a_agent,:]
                bias_W2[a_agent] = bias_W2[a_agent] + eta*gd2[a_agent]

                gd1 = x1*(1-x1)*np.dot(W2.T, gd2)
                W1 = W1 + eta*np.outer(gd1, X)
                bias_W1 = bias_W1 + eta*gd1
                               
                
                # save VARIABLES
                R_save[n]=np.copy(R)
                N_moves_save[n]=np.copy(i)
                
                break

            # IF THE EPISODE IS NOT OVER...
            h1_next = np.dot(W1, X_next) + bias_W1
            x1_next = 1/(1+np.exp(-h1_next))
            # Compute the Qvalues for the new state
            h2_next = np.dot(W2, x1_next)+bias_W2
            Qvalues_next = 1/(1+np.exp(-h2_next))

            # Compute the delta
            idx = np.where(allowed_a_next==1)[0]
            delta = R + gamma*np.max(Qvalues_next[idx]) - Qvalues[a_agent]
            
            # Update the weights
            gd2 = delta*Qvalues*(1-Qvalues)
            W2[a_agent,:] = W2[a_agent,:] + eta*np.outer(gd2, x1)[a_agent,:]
            bias_W2[a_agent] = bias_W2[a_agent] + eta*gd2[a_agent]

            gd1 = x1*(1-x1)*np.dot(W2.T, gd2)
            W1 = W1 + eta*np.outer(gd1, X)
            bias_W1 = bias_W1 + eta*gd1
            
                     
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE...     
            S=np.copy(S_next)
            X=np.copy(X_next)
            allowed_a=np.copy(allowed_a_next)
#             Qvalues = np.copy(Qvalues_next)
#             x1 = np.copy(x1_next)
            

            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS

    return R_save,N_moves_save
    

In [ ]:
N_episodes = 100000
R_save,N_moves_save = Q_Learning(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)

In [ ]:
print('Q-learning_Agent, Average reward:',np.mean(R_save),'Number of steps: ',np.mean(N_moves_save))

In [ ]:
R_save_avg = ema(R_save,3000)

plt.figure()
# plt.plot(np.arange(1,N_episodes+1,1),R_save)
plt.plot(np.arange(1,N_episodes+1,1),R_save_avg)
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
N_moves_avg = ema(N_moves_save,3000)
plt.figure()
plt.plot(np.arange(1,N_episodes+1,1),N_moves_avg)
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

In [ ]:
beta_list = [0.00001,0.00005,0.0001]     
gamma_list = [0.8,0.85,0.9]
result_list = []
for beta in beta_list:
    for gamma in gamma_list:

        R_save,N_moves_save = Q_Learning(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
        result_list.append(((beta,gamma),(np.mean(R_save),np.mean(N_moves_save))))
        print('QL_Agent, Average reward:',np.mean(R_save),'\nNumber of steps: ',np.mean(N_moves_save),
              '\nparameters: beta:',beta,'gamma:',gamma)


### Q-Learning with Adam

In [ ]:
# TRAINING LOOP BONE STRUCTURE...
# I WROTE FOR YOU A RANDOM AGENT (THE RANDOM AGENT WILL BE SLOWER TO GIVE CHECKMATE THAN AN OPTIMISED ONE, 
# SO DON'T GET CONCERNED BY THE TIME IT TAKES), CHANGE WITH YOURS ...
def QL_Adam(epsilon_0,beta,gamma,new_reward,new_feature_k1,new_feature_q1):
    S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)
    
    N_a=np.shape(allowed_a)[0]   # TOTAL NUMBER OF POSSIBLE ACTIONS
    N_in=np.shape(X)[0]    ## INPUT SIZE
    N_h=200                ## NUMBER OF HIDDEN NODES
    np.random.seed(101)
    
    R_save = np.zeros([N_episodes, 1])
    N_moves_save = np.zeros([N_episodes, 1])


    W1_A = np.random.randn(N_h, N_in) * np.sqrt(1 / (N_in))
    W2_A = np.random.randn(N_a, N_h) * np.sqrt(1 / (N_h))
    
    bias_W1_A = np.zeros((N_h,))
    bias_W2_A = np.zeros((N_a,))
    
    # Intialise Adam for the parameters
    Adam_W1=Adam(W1_A,beta1)
    Adam_W2=Adam(W2_A,beta1)
    Adam_bias_W1=Adam(bias_W1_A,beta1)
    Adam_bias_W2=Adam(bias_W2_A,beta1)

    for n in range(N_episodes):

        epsilon_f = epsilon_0 / (1 + beta * n)   ## DECAYING EPSILON
        Done=0                                   ## SET DONE TO ZERO (BEGINNING OF THE EPISODE)
        i = 1                                    ## COUNTER FOR NUMBER OF ACTIONS

        S,X,allowed_a=env.Initialise_game(new_feature_k1,new_feature_q1)      ## INITIALISE GAME
        print("N_episodes",n)                    ## REMOVE THIS OF COURSE, WE USED THIS TO CHECK THAT IT WAS RUNNING

        h1 = np.dot(W1_A, X) + bias_W1_A
        x1 = 1/(1+np.exp(-h1))
        h2 = np.dot(W2_A, x1)+bias_W2_A
        Qvalues = 1/(1+np.exp(-h2))
        
        while Done==0:                           ## START THE EPISODE
            
            

            a_agent = EpsilonGreedy_Policy(Qvalues, epsilon_f, allowed_a)

            S_next, X_next, allowed_a_next, R, Done = env.OneStep(a_agent,new_reward,new_feature_k1,new_feature_q1)

            ## THE EPISODE HAS ENDED, UPDATE...BE CAREFUL, THIS IS THE LAST STEP OF THE EPISODE
            if Done==1:
                delta = R-Qvalues[a_agent]
                                
                # Update the weights
                gd2 = delta*Qvalues*(1-Qvalues)
                W2_A[a_agent,:] = W2_A[a_agent,:] + eta*Adam_W2.Compute(np.outer(gd2, x1))[a_agent,:]
                bias_W2_A[a_agent] = bias_W2_A[a_agent] + eta*Adam_bias_W2.Compute(gd2)[a_agent]

                gd1 = x1*(1-x1)*np.dot(W2_A.T, gd2)
                W1_A = W1_A + eta*Adam_W1.Compute(np.outer(gd1, X))
                bias_W1_A = bias_W1_A + eta*Adam_bias_W1.Compute(gd1)
                               
                
                # save VARIABLES
                R_save[n]=np.copy(R)
                N_moves_save[n]=np.copy(i)
                
                break

            # IF THE EPISODE IS NOT OVER...
            h1_next = np.dot(W1_A, X_next) + bias_W1_A
            x1_next = 1/(1+np.exp(-h1_next))
            # Compute the Qvalues for the new state
            h2_next = np.dot(W2_A, x1_next)+bias_W2_A
            Qvalues_next = 1/(1+np.exp(-h2_next))

            # Compute the delta
            idx = np.where(allowed_a_next==1)[0]
            delta = R + gamma*np.max(Qvalues_next[idx]) - Qvalues[a_agent]

            
            # Update the weights
            gd2 = delta*Qvalues*(1-Qvalues)
            W2_A[a_agent,:] = W2_A[a_agent,:] + eta*Adam_W2.Compute(np.outer(gd2, x1))[a_agent,:]
            bias_W2_A[a_agent] = bias_W2_A[a_agent] + eta*Adam_bias_W2.Compute(gd2)[a_agent]

            gd1 = x1*(1-x1)*np.dot(W2_A[a_agent,:], gd2[a_agent])
            W1_A = W1_A + eta*Adam_W1.Compute(np.outer(gd1, X))
            bias_W1_A = bias_W1_A + eta*Adam_bias_W1.Compute(gd1)
            
                     
            
            # NEXT STATE AND CO. BECOME ACTUAL STATE...     
            S=np.copy(S_next)
            X=np.copy(X_next)
            allowed_a=np.copy(allowed_a_next)
            Qvalues = np.copy(Qvalues_next)
            x1 = np.copy(x1_next)
            

            i += 1  # UPDATE COUNTER FOR NUMBER OF ACTIONS

    return R_save,N_moves_save
    

In [ ]:
N_episodes = 100000
R_ql_adam, N_moves_ql_adam = QL_Adam(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)

In [ ]:
print('QL_Adam_Agent, Average reward:',np.mean(R_save),'Number of steps: ',np.mean(N_moves_save))

In [ ]:
R_save_avg = ema(R_ql_adam, 1000)

plt.figure()
plt.plot(np.arange(1,N_episodes+1,1),R_save_avg)
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
N_moves_avg = ema(N_moves_ql_adam,1000)

plt.figure()
plt.plot(np.arange(1,N_episodes+1,1), N_moves_avg)
plt.xlabel('Number of steps')
plt.ylabel('Reward')
plt.figure(figsize=(10, 6.5))
plt.show()

In [ ]:
N_episodes = 50000
beta_list = [0.00001,0.00005,0.0001]     
gamma_list = [0.8,0.85,0.9]
result_list = []
for beta in beta_list:
    for gamma in gamma_list:

        R_save,N_moves_save = QL_Adam(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
        result_list.append(((beta,gamma),(np.mean(R_save),np.mean(N_moves_save))))
        print('QL_Agent, Average reward:',np.mean(R_save),'\nNumber of steps: ',np.mean(N_moves_save),
              '\nparameters: beta:',beta,'gamma:',gamma)

## New State Representation

In [ ]:
N_episodes = 100000
R_save,N_moves_save,(best_R,best_episode),R_mean_list,N_moves_save_mean = SARSA_Adam(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
print(np.mean(R_save),np.mean(N_moves_save))

In [ ]:
plt.figure()
plt.title('New_State_k with Adam Reward Mean')
plt.plot(np.arange(100,N_episodes+1,1),R_mean_list[99:])
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()
#print(R_mean_list)

In [ ]:
plt.figure()
plt.title('New_State_k with Adam Steps Mean')
plt.plot(np.arange(100,N_episodes+1,1),N_moves_save_mean[99:])
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

print(N_moves_save_mean)

In [ ]:
R_list=ema(R_save,1000)
N_list=ema(N_moves_save,1000)
print(np.shape(R_list))

In [ ]:
plt.figure()
plt.title('New_State_q with Adam Reward Ema')
plt.plot(np.arange(0,N_episodes,1),R_list)
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()
#print(R_mean_list)

In [ ]:
plt.figure()
plt.title('New_State_q with Adam Steps Ema')
plt.plot(np.arange(0,N_episodes,1),N_list)
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

## New Reward Administration

In [ ]:
N_episodes = 100000
R_save,N_moves_save,(best_R,best_episode),R_mean_list,N_moves_save_mean = SARSA_Adam(epsilon_0,beta,gamma,new_reward=False,new_feature_k1=False,new_feature_q1=False)
print(np.mean(R_save),np.mean(N_moves_save))

In [ ]:
plt.figure()
plt.title('New_Reward Reward Mean')
plt.plot(np.arange(100,N_episodes+1,1),R_mean_list[99:])
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
plt.figure()
plt.title('New_Reward Steps Mean')
plt.plot(np.arange(100,N_episodes+1,1),N_moves_save_mean[99:])
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()

In [ ]:
R_list=ema(R_save,1000)
N_list=ema(N_moves_save,1000)
print(np.shape(R_list))

In [ ]:
plt.figure()
plt.title('New_Reward Reward Ema')
plt.plot(np.arange(0,N_episodes,1),R_list)
plt.xlabel('Episode Number')
plt.ylabel('Reward')
plt.figure(figsize=(20, 10))
plt.show()

In [ ]:
plt.figure()
plt.title('New_Reward Steps Ema')
plt.plot(np.arange(0,N_episodes,1),N_list)
plt.xlabel('Episode Number')
plt.ylabel('Number of steps')
plt.figure(figsize=(10, 6.5))
plt.show()